In [1]:
#submission

import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
import tqdm
from RandAugment import RandAugment

In [2]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = '/opt/ml/input/data/eval'

In [3]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [4]:
class ResBlock(nn.Module):
    def __init__(self,num_channels=16):
        super(ResBlock,self).__init__()
        
        self.conv1 = nn.Conv2d(num_channels,num_channels,kernel_size=3,stride=1,padding=1)
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.leakyrelu = nn.LeakyReLU(negative_slope=0.2,inplace=True)
        
        self.conv2 = nn.Conv2d(num_channels,num_channels,kernel_size=3,stride=1,padding=1)
        self.bn2 = nn.BatchNorm2d(num_channels)
    
    def forward(self,x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.leakyrelu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += residual
        return out

def make_block(r,n):
    residual = []
    
    for i in range(r):
        block = ResBlock(num_channels=n)
        residual.append(block)
    
    return nn.Sequential(*residual)

class ResizingNetwork(nn.Module):
    def __init__(self,r=1, n=16):
        super(ResizingNetwork, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=n,kernel_size=7,stride=1,padding=3)
        self.leakyrelu1 = nn.LeakyReLU(negative_slope=0.2,inplace=True)
        
        self.conv2 = nn.Conv2d(n,n,kernel_size=1,stride=1)
        self.leakyrelu2 = nn.LeakyReLU(negative_slope=0.2,inplace=True)
        self.bn1 = nn.BatchNorm2d(n)
        
        
        self.resblock = make_block(r,n)
        
        self.conv3 = nn.Conv2d(n,n,kernel_size=3,stride=1,padding=1)
        self.bn2 = nn.BatchNorm2d(n)
        
        self.conv4 = nn.Conv2d(n,out_channels=3,kernel_size=7,stride=1,padding=3)
        
    
    def forward(self,x):
        
        residual = F.interpolate(x,scale_factor=0.5,mode='bilinear',align_corners=True,recompute_scale_factor=True)
        #residual = x
        
        out = self.conv1(x)
        out = self.leakyrelu1(out)        
        out = self.conv2(out)
        out = self.leakyrelu2(out)
        out = self.bn1(out)
        
        out_residual = F.interpolate(out,scale_factor=0.5,mode='bilinear',align_corners=True,recompute_scale_factor=True)
        #out_residual = out

        out = self.resblock(out_residual)
        out = self.conv3(out)
        out = self.bn2(out)
        out += out_residual
        out = self.conv4(out)
        out += residual
        return out

In [56]:
class Mymodel(nn.Module):
    def __init__(self,resizer,recognition):
        super(Mymodel,self).__init__()
        self.resizer = resizer
        self.recognition = recognition
    
    def forward(self,x):
        resize_img = self.resize(x)
        output = self.recognition(resize_img)
        return output

In [5]:
class Maskmodel(nn.Module):
    def __init__(self,resizer,recognition):
        super(Maskmodel,self).__init__()
        self.resizer = resizer
        self.recognition = recognition
    
    def forward(self,x):
        resize_img = self.resizer(x)
        mask_output = self.recognition(resize_img)
        return mask_output

In [6]:
class Gendermodel(nn.Module):
    def __init__(self,resizer,recognition):
        super(Gendermodel,self).__init__()
        self.resizer = resizer
        self.recognition = recognition
    
    def forward(self,x):
        resize_img = self.resizer(x)
        gender_output = self.recognition(resize_img)
        return gender_output

In [7]:
class Agemodel(nn.Module):
    def __init__(self,resizer,recognition):
        super(Agemodel,self).__init__()
        self.resizer = resizer
        self.recognition = recognition
    
    def forward(self,x):
        resize_img = self.resizer(x)
        age_output = self.recognition(resize_img)
        return age_output

In [6]:
#my model
model = torch.load('efficientb6_cutmix.pth')

In [7]:
model.load_state_dict(torch.load('result_018_accuracy_99.10%.ckpt'))

<All keys matched successfully>

In [8]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

transform = transforms.Compose([
    ToTensor()
])

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]

dataset_test = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset_test,
    shuffle=False
)


In [ ]:
# basic inference
device = torch.device('cuda')

model.to(device)

model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in tqdm.tqdm(loader):
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')

In [37]:
#test time augmentation

transform = transforms.Compose([
    ToTensor()
])

transform.transforms.insert(0, RandAugment(4, 9))

dataset_test2 = TestDataset(image_paths, transform)

loader2 = DataLoader(
    dataset_test2,
    shuffle=False
)


In [38]:
transform = transforms.Compose([
    ToTensor()
])

transform.transforms.insert(0, RandAugment(4, 9))

dataset_test3 = TestDataset(image_paths, transform)

loader3 = DataLoader(
    dataset_test3,
    shuffle=False
)


In [44]:
transform = transforms.Compose([
    ToTensor()
])

dataset_test4 = TestDataset(image_paths, transform)

loader4 = DataLoader(
    dataset_test4,
    shuffle=False
)

In [ ]:
#test time augmentation prediction
device = torch.device('cuda')

model.to(device)

model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []

for images,images2,images3,images4 in tqdm.tqdm(zip(loader,loader2,loader3,loader4)):
    with torch.no_grad():
        images = images.to(device)
        images2 = images2.to(device)
        images3 = images3.to(device)
        images4 = images4.to(device)
        pred1 = F.softmax(model(images),dim=-1)
        pred2 = F.softmax(model(images2),dim=-1)
        pred3 = F.softmax(model(images3),dim=-1)
        pred4 = F.softmax(model(images4),dim=-1)
        pred = (pred1 + pred2 + pred3 + pred4) / 4
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')

In [10]:
# mask&gender&age prediction

device = torch.device('cuda')

model.to(device)
model2.to(device)
model3.to(device)

model.eval()
model2.eval()
model3.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in tqdm.tqdm(loader):
    with torch.no_grad():
        images = images.to(device)
        pred_mask = model(images)
        pred_mask = pred_mask.argmax(dim=-1)
        pred_gender = model2(images)
        pred_gender = pred_gender.argmax(dim=-1)
        pred_age = model3(images)
        pred_age = pred_age.argmax(dim=-1)
        pred = 6*pred_mask + 3*pred_gender + pred_age
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')        

100%|██████████| 12600/12600 [41:07<00:00,  5.11it/s] 

test inference is done!


In [11]:
#ensemble soft voting with two model
device = torch.device('cuda')

model.to(device)
model2.to(device)

model.eval()
model2.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []

for images in tqdm.tqdm(loader):
    with torch.no_grad():
        images = images.to(device)
        pred1 = F.softmax(model(images),dim=-1)
        pred2 = F.softmax(model2(images),dim=-1)
        pred = (pred1 + pred2) / 2
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')

100%|██████████| 12600/12600 [42:27<00:00,  4.95it/s]

test inference is done!


In [16]:
#ensemble hard voting with many model
from collections import Counter

device = torch.device('cuda')

model.to(device)
model2.to(device)
model3.to(device)
model4.to(device)
model5.to(device)
model6.to(device)

model.eval()
model2.eval()
model3.eval()
model4.eval()
model5.eval()
model6.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
voting_list = []

for images in tqdm.tqdm(loader):
    with torch.no_grad():
        images = images.to(device)
        pred1 = model(images)
        pred1 = pred1.argmax(dim=-1)
        voting_list.append(pred1)
        pred2 = model2(images)
        pred2 = pred2.argmax(dim=-1)
        voting_list.append(pred2)
        pred3 = model3(images)
        pred3 = pred3.argmax(dim=-1)
        voting_list.append(pred3)
        pred4 = model4(images)
        pred4 = pred4.argmax(dim=-1)
        voting_list.append(pred4)
        pred5 = model5(images)
        pred5 = pred5.argmax(dim=-1)
        voting_list.append(pred5)
        pred6 = model6(images)
        pred6 = pred6.argmax(dim=-1)
        voting_list.append(pred6)
        pred = Counter(voting_list).most_common(1)[0][0]
        all_predictions.extend(pred.cpu().numpy())
        voting_list = []
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')

100%|██████████| 12600/12600 [56:22<00:00,  3.73it/s] 

test inference is done!
